In [2]:
#imports
import csv
import requests
from collections import deque
import os

In [3]:
# Read the API key from a file and remove any leading/trailing whitespace.
with open('../API_Keys/NOAA_Token.txt', 'r') as file:
    api_key = file.read().strip()

# Base URL for the API calls.
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/'

# Headers required for the API call, including the authorization token.
headers = {
    'token': api_key  # API key is passed as a token in the header.
}

def api_call(url, endpoint, headers, parameters):
    """
    Make an API call to the specified URL and endpoint with given headers and parameters.
    
    Args:
        url (str): The base URL for the API.
        endpoint (str): The specific endpoint to access data from the API.
        headers (dict): Headers to include in the request (e.g., authorization tokens).
        parameters (dict): Query parameters to customize the request.
    
    Returns:
        dict: The JSON response from the API if the call is successful, None otherwise.
    """
    try:
        print(parameters['offset'])  # Debugging: print the current offset before making the call.
        response = requests.get(url + endpoint, headers=headers, params=parameters)  # Perform the GET request.
        response.raise_for_status()  # Check for HTTP errors and raise exceptions for them.
        print("API Called")  # Debugging: confirm the API was called.
        return response.json()  # Return the parsed JSON response.
    except requests.exceptions.RequestException as e:
        print(f"API call failed: {e}")  # Handle exceptions (e.g., network issues, 4xx and 5xx errors).
        return None

def append_to_csv(file_path, data, fieldnames):
    """
    Append data to a CSV file. If the file doesn't exist, create it and write the headers.
    
    Args:
        file_path (str): Path to the CSV file.
        data (list): A list of dictionaries representing the rows to append.
        fieldnames (list): Headers or fieldnames for the CSV.
    """
    with open(file_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if file.tell() == 0:  # Check if the file is empty to decide if headers need to be written.
            writer.writeheader()
        writer.writerows(data)  # Append the data rows to the CSV file.

def append_metadata_to_csv(metadata_file_path, metadata):
    """
    Append metadata to a CSV file.
    
    Args:
        metadata_file_path (str): Path to the CSV file where metadata is stored.
        metadata (dict): Metadata to be appended.
    """
    with open(metadata_file_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=metadata['resultset'].keys())
        if file.tell() == 0:  # Check if the file is empty to decide if headers need to be written.
            writer.writeheader()
        writer.writerow(metadata['resultset'])  # Append metadata to the metadata CSV.
        
def execute_api_calls_with_retries(base_url, endpoint, headers, initial_parameters, file_path, metadata_file_path, error_log_path, max_retries=3):
    """
    Execute API calls with a mechanism for retries on failure, appending successful results to a CSV file
    and metadata to another CSV file.
    
    Args:
        base_url (str): The base URL for the API calls.
        endpoint (str): The specific endpoint to access data from the API.
        headers (dict): Headers to include in the request.
        initial_parameters (dict): Initial query parameters for the API call.
        file_path (str): Path to the CSV file where results are stored.
        metadata_file_path (str): Path to the CSV file where metadata is stored.
        error_log_path (str): Path to the CSV file where failed attempts are logged.
        max_retries (int): Maximum number of retry attempts for a failed API call.
    """
    if os.path.exists(file_path):
        print(f"CSV file {file_path} already exists. Operation cancelled to prevent overwriting.")
        return  # Prevents overwriting existing data by aborting if the file already exists.
    
    stack = deque([initial_parameters])  # Use a stack to manage API call parameters, starting with the initial parameters.
    failed_attempts = []  # Track parameters that fail to succeed after the maximum number of retries.
    
    while stack:
        parameters = stack.pop()  # Pop the last set of parameters to make an API call.
        retries = parameters.pop('retries', 0)  # Extract or initialize the retry counter for these parameters.
        data = api_call(base_url, endpoint, headers, parameters)  # Attempt the API call.
        
        if data and 'results' in data:  # Check if the call was successful and data was returned.
            append_to_csv(file_path, data['results'], data['results'][0].keys())  # Append successful results to the CSV.
            append_metadata_to_csv(metadata_file_path, data['metadata'])  # Append metadata to the metadata CSV
            
            # Check if there are more results to fetch and prepare the next set of parameters.
            if parameters.get('offset', 1) + parameters.get('limit', 10) - 1 < data['metadata']['resultset']['count']:
                next_params = parameters.copy()
                next_params['offset'] += parameters.get('limit', 10)
                stack.append(next_params)  # Add the next parameters to the stack for subsequent API calls.
        else:
            # Retry logic for failed attempts.
            if retries < max_retries:
                parameters['retries'] = retries + 1  # Increment the retry counter.
                stack.append(parameters)  # Re-add the parameters to the stack for retry.
            else:
                print(f"Max retries reached for parameters: {parameters}")
                failed_attempts.append(parameters)  # Log parameters that exceeded retry attempts.
    
    # Optionally log failed attempts to a specified CSV for troubleshooting.
    if failed_attempts:
        append_to_csv(error_log_path, failed_attempts, ['locationid', 'limit', 'offset', 'retries'])

In [5]:
# Example of how to call the function
endpoint = "/stations/"
parameters = {
    "locationid": "FIPS:27",
    "limit": 1000,
    "offset": 1
}
file_path = "stations.csv"
error_log_path = "error_log.csv"
metadata_path = "stations_meta.csv"

execute_api_calls_with_retries(url, endpoint, headers, parameters, file_path, metadata_path, error_log_path, max_retries=3)

1
API Called
1001
API Called
2001
API Called


In [11]:
#Test downloading data from all stations
endpoint = '/data/'

#Get list of station codes for the Minnesota stations
parameters = {
    'datasetid': 'GHCND',
    'locationid': 'FIPS:27',
    'limit': 100, # max is 1000 for a single request
    'offset': 498801,
    'startdate': '2023-01-01',
    'enddate': '2023-12-31'
}

file_path = "data_498801_on.csv"
error_log_path = "error_log.csv"

execute_api_calls_with_retries(url, endpoint, headers, parameters, file_path, error_log_path, max_retries=3)

498801
API Called
498901
API Called
499001
API Called
499101
API Called
499201
API call failed: 503 Server Error: Service Unavailable for url: https://www.ncdc.noaa.gov/cdo-web/api/v2//data/?datasetid=GHCND&locationid=FIPS%3A27&limit=100&offset=499201&startdate=2023-01-01&enddate=2023-12-31
499201
API Called
499301
API Called
499401
API Called
499501
API Called
499601
API Called
499701
API Called
499801
API Called
499901
API Called
500001
API Called
500101
API Called
500201
API Called
500301
API Called
500401
API Called
500501
API Called
500601
API Called
500701
API Called
500801
API Called
500901
API Called
501001
API Called
501101
API Called
501201
API Called
501301
API Called
501401
API Called
501501
API Called
501601
API Called
501701
API Called
501801
API call failed: 503 Server Error: Service Unavailable for url: https://www.ncdc.noaa.gov/cdo-web/api/v2//data/?datasetid=GHCND&locationid=FIPS%3A27&limit=100&offset=501801&startdate=2023-01-01&enddate=2023-12-31
501801
API Called
50

API Called
533401
API Called
533501
API Called
533601
API Called
533701
API Called
533801
API Called
533901
API Called
534001
API Called
534101
API Called
534201
API Called
534301
API Called
534401
API Called
534501
API Called
534601
API Called
534701
API Called
534801
API Called
534901
API Called
535001
API Called
535101
API Called
535201
API Called
535301
API Called
535401
API Called
535501
API call failed: 503 Server Error: Service Unavailable for url: https://www.ncdc.noaa.gov/cdo-web/api/v2//data/?datasetid=GHCND&locationid=FIPS%3A27&limit=100&offset=535501&startdate=2023-01-01&enddate=2023-12-31
535501
API Called
535601
API Called
535701
API call failed: 503 Server Error: Service Unavailable for url: https://www.ncdc.noaa.gov/cdo-web/api/v2//data/?datasetid=GHCND&locationid=FIPS%3A27&limit=100&offset=535701&startdate=2023-01-01&enddate=2023-12-31
535701
API Called
535801
API Called
535901
API Called
536001
API Called
536101
API Called
536201
API Called
536301
API call failed: 503

API Called
569201
API Called
569301
API Called
569401
API Called
569501
API Called
569601
API Called
569701
API Called
569801
API Called
569901
API Called
570001
API Called
570101
API Called
570201
API Called
570301
API Called
570401
API Called
570501
API Called
570601
API Called
570701
API Called
570801
API Called
570901
API Called
571001
API Called
571101
API Called
571201
API Called
571301
API Called
571401
API call failed: 503 Server Error: Service Unavailable for url: https://www.ncdc.noaa.gov/cdo-web/api/v2//data/?datasetid=GHCND&locationid=FIPS%3A27&limit=100&offset=571401&startdate=2023-01-01&enddate=2023-12-31
571401
API Called
571501
API Called
571601
API Called
571701
API call failed: 503 Server Error: Service Unavailable for url: https://www.ncdc.noaa.gov/cdo-web/api/v2//data/?datasetid=GHCND&locationid=FIPS%3A27&limit=100&offset=571701&startdate=2023-01-01&enddate=2023-12-31
571701
API Called
571801
API Called
571901
API Called
572001
API Called
572101
API Called
572201
AP

API Called
606201
API Called
606301
API Called
606401
API Called
606501
API Called
606601
API Called
606701
API Called
606801
API Called
606901
API Called
607001
API Called
607101
API Called
607201
API Called
607301
API Called
607401
API Called
607501
API Called
607601
API Called
607701
API Called
607801
API Called
607901
API Called
608001
API Called
608101
API Called
608201
API Called
608301
API Called
608401
API Called
608501
API Called
608601
API call failed: 503 Server Error: Service Unavailable for url: https://www.ncdc.noaa.gov/cdo-web/api/v2//data/?datasetid=GHCND&locationid=FIPS%3A27&limit=100&offset=608601&startdate=2023-01-01&enddate=2023-12-31
608601
API Called
608701
API Called
608801
API Called
608901
API Called
609001
API Called
609101
API Called
609201
API Called
609301
API Called
609401
API Called
609501
API Called
609601
API Called
609701
API Called
609801
API Called
609901
API Called
610001
API Called
610101
API Called
610201
API Called
610301
API Called
610401
API Ca

API Called
645201
API Called
645301
API Called
645401
API Called
645501
API Called
645601
API Called
645701
API Called
645801
API Called
645901
API Called
646001
API Called
646101
API Called
646201
API Called
646301
API Called
646401
API Called
646501
API Called
646601
API Called
646701
API Called
646801
API Called
646901
API Called
647001
API Called
647101
API Called
647201
API Called
647301
API Called
647401
API Called
647501
API Called
647601
API Called
647701
API Called
647801
API Called
647901
API Called
648001
API Called
648101
API Called
648201
API Called
648301
API Called
648401
API Called
648501
API Called
648601
API Called
648701
API Called
648801
API Called
648901
API Called
649001
API Called
649101
API Called
649201
API Called
649301
API Called
649401
API Called
649501
API Called
649601
API Called
649701
API Called
649801
API Called
649901
API Called
650001
API Called
650101
API Called
650201
API Called
650301
API Called
650401
API Called
650501
API Called
650601
API Called

...

API Called
720401
API Called
720501
API Called
720601
API Called
720701
API Called
720801
API Called
720901
API Called
721001
API Called
721101
API Called
721201
API Called
721301
API Called
721401
API Called
721501
API Called
721601
API Called
721701
API Called
721801
API Called
721901
API Called
722001
API Called
722101
API Called
722201
API Called
722301
API Called
722401
API Called
722501
API Called
722601
API Called
722701
API Called
722801
API Called
722901
API Called
723001
API Called
723101
API Called
723201
API Called
723301
API Called
723401
API Called
723501
API Called
723601
API Called
723701
API Called
723801
API Called
723901
API Called
724001
API Called
724101
API Called
724201
API Called
724301
API Called
724401
API Called
724501
API Called
724601
API Called
724701
API Called
724801
API Called
724901
API Called
725001
API Called
725101
API Called
725201
API Called
725301
API Called
725401
API Called
725501
API Called
725601
API Called
725701
API Called
725801
API Called

API Called
759901
API Called
760001
API Called
760101
API Called
760201
API Called
760301
API Called
760401
API Called
760501
API Called
760601
API Called
760701
API Called
760801
API Called
760901
API Called
761001
API Called
761101
API Called
761201
API Called
761301
API Called
761401
API Called
761501
API Called
761601
API Called
761701
API Called
761801
API Called
761901
API Called
762001
API Called
762101
API Called
762201
API Called
762301
API Called
762401
API Called
762501
API Called
762601
API Called
762701
API Called
762801
API Called
762901
API Called
763001
API Called
763101
API Called
763201
API Called
763301
API Called
763401
API Called
763501
API Called
763601
API Called
763701
API Called
763801
API Called
763901
API Called
764001
API Called
764101
API Called
764201
API Called
764301
API Called
764401
API Called
764501
API Called
764601
API Called
764701
API Called
764801
API Called
764901
API Called
765001
API Called
765101
API Called
765201
API Called
765301
API Called

API Called
792201
API Called
792301
API Called
792401
API Called
792501
API Called
792601
API call failed: 503 Server Error: Service Unavailable for url: https://www.ncdc.noaa.gov/cdo-web/api/v2//data/?datasetid=GHCND&locationid=FIPS%3A27&limit=100&offset=792601&startdate=2023-01-01&enddate=2023-12-31
792601
API Called
792701
API Called
792801
API Called
792901
API Called
793001
API Called
793101
API Called
793201
API Called
793301
API Called
793401
API Called
793501
API Called
793601
API Called
793701
API Called
793801
API Called
793901
API Called
794001
API Called
794101
API Called
794201
API Called
794301
API Called
794401
API Called
794501
API Called
794601
API Called
794701
API Called
794801
API Called
794901
API Called
795001
API Called
795101
API Called
795201
API Called
795301
API Called
795401
API Called
795501
API Called
795601
API Called
795701
API Called
795801
API Called
795901
API Called
796001
API Called
796101
API Called
796201
API Called
796301
API Called
796401
API Ca

API Called
826101
API Called
826201
API Called
826301
API Called
826401
API Called
826501
API Called
826601
API Called
826701
API Called
826801
API Called
826901
API Called
827001
API Called
827101
API Called
827201
API Called
827301
API Called
827401
API Called
827501
API Called
827601
API Called
827701
API Called
827801
API Called
827901
API Called
828001
API Called
828101
API Called
828201
API Called
828301
API Called
828401
API Called
828501
API Called
828601
API Called
828701
API Called
828801
API Called
828901
API Called
829001
API Called
829101
API Called
829201
API Called
829301
API Called
829401
API Called
829501
API Called
829601
API Called
829701
API Called
829801
API Called
829901
API Called
830001
API Called
830101
API Called
830201
API Called
830301
API Called
830401
API Called
830501
API Called
830601
API Called
830701
API Called
830801
API Called
830901
API Called
831001
API Called
831101
API Called
831201
API Called
831301
API Called
831401
API Called
831501
API Called

API Called
868001
API Called
868101
API Called
868201
API Called
868301
API Called
868401
API Called
868501
API Called
868601
API Called
868701
API Called
868801
API Called
868901
API Called
869001
API Called
869101
API Called
869201
API Called
869301
API Called
869401
API Called
869501
API Called
869601
API Called
869701
API Called
869801
API Called
869901
API Called
870001
API Called
870101
API Called
870201
API Called
870301
API Called
870401
API Called
870501
API Called
870601
API Called
870701
API Called
870801
API Called
870901
API Called
871001
API Called
871101
API Called
871201
API Called
871301
API Called
871401
API Called
871501
API Called
871601
API Called
871701
API Called
871801
API Called
871901
API call failed: 503 Server Error: Service Unavailable for url: https://www.ncdc.noaa.gov/cdo-web/api/v2//data/?datasetid=GHCND&locationid=FIPS%3A27&limit=100&offset=871901&startdate=2023-01-01&enddate=2023-12-31
871901
API Called
872001
API Called
872101
API Called
872201
API Ca